In [10]:
import pandas as pd
import jwt
import requests
import datetime
from datetime import date, timedelta
import gzip
from io import StringIO
from dotenv import load_dotenv
import os


In [11]:
def configure():
    load_dotenv()

configure()

In [12]:
APPLE_CONNECT_PRIVATE_KEY = os.getenv('APPLE_CONNECT_PRIVATE_KEY')
APPLE_CONNECT_ISSUER_ID = os.getenv('APPLE_CONNECT_ISSUER_ID')
APPLE_CONNECT_APP_ID = os.getenv('APPLE_CONNECT_APP_ID')
APPLE_CONNECT_KEY_ID = os.getenv('APPLE_CONNECT_KEY_ID')
APPLE_CONNECT_VENDOR_NUMBER = os.getenv('APPLE_CONNECT_VENDOR_NUMBER')

In [13]:
def days_in_month(year: int, month: int) -> list:
    #Check if the month is valid
    if not (1 <= month <= 12):
        return f"Error: Invalid month '{month}'. Please enter a month between 1 and 12."

    #Start with the first day of the month
    start_date = date(year, month, 1)
    
    #Calculate the next month's first day to determine the end of the current month
    if month == 12:
        next_month = date(year + 1, 1, 1)
    else:
        next_month = date(year, month + 1, 1)
    
    # Use a loop to generate all dates in the month
    current_date = start_date
    days = []
    while current_date < next_month:
        # Format the date as "YYYY-M-D"
        days.append(f"{current_date.year}-{current_date.month:02d}-{current_date.day:02d}")
        current_date += timedelta(days=1)
    
    return days


In [14]:
def generate_jwt():
    if not APPLE_CONNECT_PRIVATE_KEY or not APPLE_CONNECT_ISSUER_ID or not APPLE_CONNECT_KEY_ID:
        raise ValueError("Missing environment variables: APPLE_CONNECT_PRIVATE_KEY, APPLE_CONNECT_ISSUER_ID, APPLE_CONNECT_KEY_ID.")

    # Replace literal \n with actual newlines
    private_key = APPLE_CONNECT_PRIVATE_KEY.replace("\\n", "\n")

    # JWT headers
    headers = {
        "alg": "ES256",
        "kid": APPLE_CONNECT_KEY_ID,
    }

    # JWT payload
    payload = {
        "iss": APPLE_CONNECT_ISSUER_ID,
        "exp": int((datetime.datetime.now() + datetime.timedelta(minutes=20)).timestamp()),
        "aud": "appstoreconnect-v1",
    }

    # Generate the token
    token = jwt.encode(payload, private_key, algorithm="ES256", headers=headers)
    return token

In [15]:
# OUTPUT CSV FILE
def get_data_app(days_list: list, output_file: str):

    url = "https://api.appstoreconnect.apple.com/v1/salesReports"

    # Prepare headers, including the JWT token for authentication
    headers = {
        "Authorization": f"Bearer {generate_jwt()}",
        "Content-Type": "application/json",
    }

    # Open the output file in write mode
    with open(output_file, 'w', newline='', encoding='utf-8') as f:
        # Loop through the days list and fetch data
        for day in days_list:
            print(f"Data for {day} is loading")
            try:
                # Define parameters for the request for the specified day
                params = {
                    "filter[reportDate]": day,  # Filter sales data for the specified day
                    "filter[vendorNumber]": "91178476",  # Vendor Number (replace with your actual vendor number)
                    "filter[reportType]": "SALES",  # Report type could be 'SUMMARY', 'DETAILED', etc.
                    "filter[reportSubType]": "SUMMARY",
                    "filter[frequency]": "DAILY",  # Frequency of the report (DAILY)
                }

                # Send the request to the API for the current day
                response = requests.get(url, headers=headers, params=params)

                # Check if the response was successful
                response.raise_for_status()  # Raise an error for bad responses (4xx or 5xx)

                # Decompress and decode the content from the response
                contents = gzip.decompress(response.content)
                decompressed_data_str = contents.decode("utf-8")

                # Convert the decompressed string data to a CSV string
                data = StringIO(decompressed_data_str)

                # Write the CSV data to the output file (append mode for subsequent days)
                

                if f.tell() == 0:  # Check if the file is empty (write header if it's the first write)
                    # Write the CSV header based on the first day's data
                    for line in data:
                        f.write(line)
                    print(f"Data for {day} was saved to {output_file}.")
                else:
                    # Skip the header for subsequent days
                    next(data)  # Skip the first line (header)
                    for line in data:
                        f.write(line)
                    print(f"Data for {day} was appended to {output_file}.")

            except requests.exceptions.RequestException as e:
                # Handle any HTTP request issues
                print(f"Failed to fetch data for {day}: {e}")
                continue  # Skip this day and continue with the next one

            except gzip.error as e:
                # Handle any GZIP decompression issues
                print(f"Failed to decompress data for {day}: {e}")
                continue  # Skip this day and continue with the next one

            except Exception as e:
                # Handle any other unexpected errors
                print(f"An error occurred for {day}: {e}")
                continue  # Skip this day and continue with the next one

    print(f"All available data has been saved to {output_file}.")


In [16]:
year = 2024 # Select Year
month = 12 # Select Month
output_file = "appstore_sales_data.csv"
days_list = days_in_month(year, month)
get_data_app(days_list, output_file)

Data for 2024-12-01 is loading
Data for 2024-12-01 was saved to appstore_sales_data.csv.
Data for 2024-12-02 is loading
Data for 2024-12-02 was appended to appstore_sales_data.csv.
Data for 2024-12-03 is loading
Data for 2024-12-03 was appended to appstore_sales_data.csv.
Data for 2024-12-04 is loading
Data for 2024-12-04 was appended to appstore_sales_data.csv.
Data for 2024-12-05 is loading
Data for 2024-12-05 was appended to appstore_sales_data.csv.
Data for 2024-12-06 is loading
Data for 2024-12-06 was appended to appstore_sales_data.csv.
Data for 2024-12-07 is loading
Data for 2024-12-07 was appended to appstore_sales_data.csv.
Data for 2024-12-08 is loading
Data for 2024-12-08 was appended to appstore_sales_data.csv.
Data for 2024-12-09 is loading
Data for 2024-12-09 was appended to appstore_sales_data.csv.
Data for 2024-12-10 is loading
Data for 2024-12-10 was appended to appstore_sales_data.csv.
Data for 2024-12-11 is loading
Data for 2024-12-11 was appended to appstore_sales_d

In [17]:
# # OUTPUT PANDAS DATAFRAME
# def get_data_app(days_list: list):

#     url = "https://api.appstoreconnect.apple.com/v1/salesReports"

#     # Prepare headers, including the JWT token for authentication
#     headers = {
#         "Authorization": f"Bearer {generate_jwt()}",
#         "Content-Type": "application/json",
#     }

#     # Initialize an empty list to collect DataFrames
#     dfs = []
#     for day in days_list:
#         print(f"Data for {day} is loading")
#         try:
#             # Define parameters for the request for the specified day
#             params = {
#                 "filter[reportDate]": day,  # Filter sales data for the specified day
#                 "filter[vendorNumber]": "91178476",  # Vendor Number (replace with your actual vendor number)
#                 "filter[reportType]": "SALES",  # Report type could be 'SUMMARY', 'DETAILED', etc.
#                 "filter[reportSubType]": "SUMMARY",
#                 "filter[frequency]": "DAILY",  # Frequency of the report (DAILY)
#             }

#             # Send the request to the API for the current day
#             response = requests.get(url, headers=headers, params=params)

#             # Check if the response was successful
#             response.raise_for_status()  # Raise an error for bad responses (4xx or 5xx)

#             # Decompress and decode the content from the response
#             contents = gzip.decompress(response.content)
#             decompressed_data_str = contents.decode("utf-8")

#             # Convert the decompressed string data to a pandas DataFrame
#             data = StringIO(decompressed_data_str)
#             df = pd.read_csv(data, sep="\t")

#             # Append the DataFrame to the list
#             dfs.append(df)
#             print(f"Data for {day} was loaded and appended to the main dataset.")

#         except requests.exceptions.RequestException as e:
#             # Handle any HTTP request issues
#             print(f"Failed to fetch data for {day}: {e}")
#             continue  # Skip this day and continue with the next one

#         except gzip.error as e:
#             # Handle any GZIP decompression issues
#             print(f"Failed to decompress data for {day}: {e}")
#             continue  # Skip this day and continue with the next one

#         except Exception as e:
#             # Handle any other unexpected errors
#             print(f"An error occurred for {day}: {e}")
#             continue  # Skip this day and continue with the next one

#     # Concatenate all DataFrames in the list into one DataFrame if there's any data
#     if dfs:
#         final_df = pd.concat(dfs, ignore_index=True)
#         return final_df
#     else:
#         print("No data was fetched.")
#         return None    

In [18]:
# year = 2024 # Select Year
# month = 12 # Select Month
# days_list = days_in_month(year, month)
# df_app = get_data_app(days_list)